In [19]:
import sys
if ".." not in sys.path:
    sys.path.append("..")

In [21]:
import warnings

import numpy as np
import pandas as pd
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
from utils.cobra_utils import get_or_create_exchange, set_active_bound

# Load the model
model = read_sbml_model("../model/Rpom_05.xml")
ex_glc = model.reactions.get_by_id("EX_glc")
ex_ace = get_or_create_exchange(model, "ACET[e]")

# TODO: Remove all of the below
# Turn on maintenance
atpm = model.reactions.get_by_id("ATPM")
# atpm.bounds = (25, 25)

https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http(s)://identifiers.org/collection/id' or'http(s)://identifiers.org/COLLECTION:id
https://identifiers.org/stem/ does not conform to 'http

In [16]:
pro_mets = {'proline' : "PRO",
 '4-aminobenzoic acid' : "P-AMINO-BENZOATE",
 'taurine' : "TAURINE",
 'glutamic acid' : "GLT",
 'alanine' : "L-ALPHA-ALANINE",
 'phenylalanine' : "PHE",
 'leucine' : "LEU",
 '2′deoxyguanosine' : "DEOXYGUANOSINE",
 'valine' : "VAL",
 'tyrosine': "TYR",}

pro_mets = {'glutamic acid' : "GLT",
 'phenylalanine' : "PHE",
 'valine' : "VAL",
 'isoleucine' : "ILE",
 'alanine' : "L-ALPHA-ALANINE",
 'leucine' : "LEU",
 'aspartic acid' : "L-ASPARTATE",
 '4-aminobenzoic acid' : "P-AMINO-BENZOATE",
 '2′deoxyguanosine' : "DEOXYGUANOSINE",
 'proline' : "PRO"}



In [24]:
micro_growth_rates = {}
for metname, metid in micro_mets.items():
    with model:
        try:
            met_e = model.metabolites.get_by_id(f"{metid}[e]")
        except KeyError:
            # Create external metabolite, allow to enter the cell
            print(f"Creating external metabolite {metid}[e] and free transport reaction")
            met_c = model.metabolites.get_by_id(f"{metid}[c]")
            met_e = Metabolite(
                id=f"{metid}[e]",
                name=metname,
                formula=met_c.formula,
                charge=met_c.charge,
                compartment="e",
            )
            model.add_metabolites([met_e])

            # Create the free transport reaction
            rxn = Reaction(f"{metid}tx")
            rxn.name = f"Free transport of {metname}"
            rxn.bounds = (0, 1000)
            rxn.add_metabolites({met_e: -1, met_c: 1})
            model.add_reactions([rxn])

        # Add the exchange reaction for the metabolite
        ex_met = get_or_create_exchange(model, met_e.id)
        ex_met.bounds = (-10, 0)

        sol = model.optimize()
    
    micro_growth_rates[metname] = sol.objective_value
micro_growth_rates

Creating external metabolite PRO[e] and free transport reaction
Creating external metabolite P-AMINO-BENZOATE[e] and free transport reaction
Creating external metabolite L-ALPHA-ALANINE[e] and free transport reaction
Creating external metabolite PHE[e] and free transport reaction
Creating external metabolite LEU[e] and free transport reaction
Creating external metabolite DEOXYGUANOSINE[e] and free transport reaction
Creating external metabolite VAL[e] and free transport reaction
Creating external metabolite TYR[e] and free transport reaction


{'proline': 0.8101985146164805,
 '4-aminobenzoic acid': 0.0,
 'taurine': 0.33651157832305956,
 'glutamic acid': 0.0,
 'alanine': 0.5497592665691878,
 'phenylalanine': 0.7405523044327131,
 'leucine': 0.789621059192629,
 '2′deoxyguanosine': 1.3237052674684842,
 'valine': 0.0,
 'tyrosine': 0.568052934582511}

In [25]:
with model:
    # model.reactions.get_by_id('BRANCHED-CHAINAMINOTRANSFERVAL-RXN').bounds = (-1000, 1000)

    # For now, supply free glutamate
    dmd = model.add_boundary(
        model.metabolites.get_by_id("GLT[c]"),
        type="demand",
        reaction_id="EX_GLT"
    )
    dmd.bounds = (-10, 0)

    sol = model.optimize()
sol

,fluxes,reduced_costs
1-ACYLGLYCEROL-3-P-ACYLTRANSFER-RXN,0.060516,0.000000e+00
1.1.1.178-RXN,0.000000,-3.218328e-02
1.1.1.262-RXN,0.000000,0.000000e+00
1.1.1.39-RXN,0.000000,-9.397412e-14
1.1.1.8-RXN,0.111859,-4.440892e-16
...,...,...
RXN-9106,0.000000,-9.367507e-17
RXN0-5258-Yeast,0.000000,1.387779e-17
RXN-19301,0.000000,-5.204170e-18
GLYC3PDEHYDROG-RXN,0.000000,1.387779e-17


In [18]:
pro_growth_rates = {}
for metname, metid in pro_mets.items():
    with model:
        try:
            met_e = model.metabolites.get_by_id(f"{metid}[e]")
        except KeyError:
            # Create external metabolite, allow to enter the cell
            print(f"Creating external metabolite {metid}[e] and free transport reaction")
            met_c = model.metabolites.get_by_id(f"{metid}[c]")
            met_e = Metabolite(
                id=f"{metid}[e]",
                name=metname,
                formula=met_c.formula,
                charge=met_c.charge,
                compartment="e",
            )
            model.add_metabolites([met_e])

            # Create the free transport reaction
            rxn = Reaction(f"{metid}tx")
            rxn.name = f"Free transport of {metname}"
            rxn.bounds = (0, 1000)
            rxn.add_metabolites({met_e: -1, met_c: 1})
            model.add_reactions([rxn])

        # Add the exchange reaction for the metabolite
        ex_met = get_or_create_exchange(model, met_e.id)
        ex_met.bounds = (-10, 0)

        sol = model.optimize()
    
    pro_growth_rates[metname] = sol.objective_value
pro_growth_rates

Creating external metabolite PHE[e] and free transport reaction
Creating external metabolite VAL[e] and free transport reaction
Creating external metabolite ILE[e] and free transport reaction
Creating external metabolite L-ALPHA-ALANINE[e] and free transport reaction
Creating external metabolite LEU[e] and free transport reaction
Creating external metabolite L-ASPARTATE[e] and free transport reaction
Creating external metabolite P-AMINO-BENZOATE[e] and free transport reaction
Creating external metabolite DEOXYGUANOSINE[e] and free transport reaction
Creating external metabolite PRO[e] and free transport reaction


{'glutamic acid': 0.0,
 'phenylalanine': 0.7405523044324391,
 'valine': 0.0,
 'isoleucine': 0.7855428752994029,
 'alanine': 0.549759266657687,
 'leucine': 0.7896210591926038,
 'aspartic acid': 0.5593198385547369,
 '4-aminobenzoic acid': 0.0,
 '2′deoxyguanosine': 1.3237052674674843,
 'proline': 0.8101985146166797}